In [1]:
# The goal of this notebook will be to separate the different data into CSVs
# which will share a common ID that can be used for SQL joins after being put
# into a Postgres instance at ElephantSQL

import pandas as pd
df = pd.read_csv('..\\src\\extraneous_datasets\\heartDisease_data_9001.csv')
df.head()

City_Name     state  population  population_change  Median_Age  \
0    Niagara Falls  New York     48460.0              -12.8        38.6   
1  North Tonawanda  New York     30475.0               -8.4        44.3   
2            Nyack  New York      7040.0                4.5        42.1   
3            Olean  New York     13711.0              -10.7        40.8   
4          Oneonta  New York     14057.0                5.8        22.1   

   Median_Income  per_capita_Income  Median_House_Value  Median_Rent  \
0        35995.0            64894.0             76050.0        665.0   
1        56169.0            64894.0            122506.0        686.0   
2        65999.0            64894.0            478868.0       1556.0   
3        42866.0            64894.0             79215.0        605.0   
4        42781.0            64894.0            157787.0        882.0   

   Cost_of_Living_Index  ...  popchangescore  MedIncscore  PerCapIncscore  \
0                 100.7  ...            42.2         12.0            50.0   
1                 101.0  ...            43.0         31.2            50.0   
2                 154.9  ...            45.2         43.8            50.0   
3                  82.4  ...            42.6         17.3            50.0   
4                  88.8  ...            45.5         17.3            50.0   

  PopulationScore  EconomyScore  ClimateScore  CostofLivingScore  \
0       46.266667     22.666667         52.33          61.183333   
1       38.033333     50.366667         52.33          76.186667   
2       25.400000     48.366667         97.84           4.886667   
3       46.300000     27.466667         75.77          81.466667   
4       65.800000     25.400000         49.49          49.623333   

   LivabilityScore  Heart Disease Value       county  
0        45.611667                215.1      Niagara  
1        54.229167                215.1      Niagara  
2        44.123333                142.9     Rockland  
3        57.750833                226.3  Cattaraugus  
4        47.578333                175.9       Otsego  

[5 rows x 41 columns]

In [2]:
# This code will show how many counties are showing up in the data
# The first Release Canvas only reguires 50% of the data to be shown
# so, at this stage, it is okay to have some missing data.

for state in sorted(df['state'].unique()):
    print(f"{state}: {df['county'][df['state']==state].unique().shape}")

Alabama: (15,)
Arizona: (10,)
Arkansas: (12,)
California: (46,)
Colorado: (13,)
Connecticut: (6,)
Delaware: (1,)
Georgia: (23,)
Hawaii: (4,)
Illinois: (6,)
Indiana: (1,)
Kansas: (9,)
Louisiana: (22,)
Maryland: (7,)
Massachusetts: (4,)
Michigan: (17,)
Mississippi: (3,)
Montana: (1,)
Nebraska: (4,)
New Hampshire: (5,)
New Jersey: (8,)
New Mexico: (10,)
New York: (21,)
North Dakota: (3,)
Ohio: (2,)
Oklahoma: (12,)
Oregon: (16,)
Pennsylvania: (15,)
Rhode Island: (2,)
South Carolina: (14,)
Tennessee: (8,)
Texas: (57,)
Utah: (4,)
Virginia: (10,)
Washington: (17,)


In [3]:
name = df[df['state'] == 'Alabama']
output = name.to_dict('records')
county = [output[i]['county'] for i in list(range(len(output)))]
stats = [output[i]['Heart Disease Value'] for i in list(range(len(output)))]
print(dict(zip(county, stats)))

{'Tuscaloosa': 232.1, 'Tallapoosa': 241.6, 'Covington': 263.4, 'Escambia': 231.3, 'Baldwin': 186.1, 'Etowah': 275.9, 'Chilton': 271.7, 'Cullman': 260.5, 'Marengo': 332.7, 'Barbour': 253.8, 'Mobile': 232.4, 'Colbert': 238.3, 'Dale': 217.8, 'Autauga': 200.1, 'Talladega': 267.4}


In [4]:
df.columns

Index(['City_Name', 'state', 'population', 'population_change', 'Median_Age',
       'Median_Income', 'per_capita_Income', 'Median_House_Value',
       'Median_Rent', 'Cost_of_Living_Index', 'Percent_below_Poverty',
       'Population_Density', 'Unemployment_rate', 'Most_Common_Industries',
       'Property_taxes', 'Latitude', 'Longitude', 'Average_Commute_Time',
       'AQI', 'popden_score', 'popden_norm', 'HDTI', 'HDTIscore',
       'HDTIscorenorm', 'RTI', 'RTIscore', 'RTIscorenorm', 'AQIscore',
       'MedAgescore', 'Povertyscore', 'COLIscore', 'popchangescore',
       'MedIncscore', 'PerCapIncscore', 'PopulationScore', 'EconomyScore',
       'ClimateScore', 'CostofLivingScore', 'LivabilityScore',
       'Heart Disease Value', 'county'],
      dtype='object')

In [5]:
# This column will between shared by all of the CSV files

df['id'] = pd.Series(list(range(len(df['City_Name']))))

In [6]:
# Use this package to work with cities/states/counties

from geonamescache import GeonamesCache as geo
import geonamescache as g

In [7]:
state_dict = geo.us_states

In [8]:
state_code = list(state_dict.keys())

In [9]:
state_name = []

for state in state_code:
    state_name.append(list(state_dict[state].values())[1])
print(state_name)

['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']


In [10]:
print(state_code)

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [11]:
code_name = dict(zip(state_name, state_code))

In [12]:
# Load in dataset containing all FIPS county codes

fips = pd.read_csv('https://raw.githubusercontent.com/kjhealy/fips-codes/'
                   'master/state_and_county_fips_master.csv')

In [13]:
l1 = []
for state in df['state']:
    if state in list(code_name.keys()):
        l1.append(code_name[state])
print(set(l1))

{'IL', 'RI', 'NJ', 'VA', 'GA', 'CO', 'AL', 'KS', 'NM', 'LA', 'PA', 'ND', 'CA', 'HI', 'AR', 'NY', 'TN', 'UT', 'NH', 'MI', 'OR', 'WA', 'MS', 'DE', 'OH', 'SC', 'MT', 'NE', 'AZ', 'MD', 'CT', 'MA', 'TX', 'OK', 'IN'}


In [14]:
assert(len(l1) == len(df['state']))

In [15]:
# Rename the columns and do some cleaning

fips['code'] = fips['state']
fips['county'] = fips['name'].str.replace(' County', '')
fips = fips.drop(columns=['state', 'name'])
fips[:1]

fips code         county
0     0  NaN  UNITED STATES

In [16]:
# Adds a column 'code' (state abbrev)

df['code'] = pd.Series(l1)
df[:1]

City_Name     state  population  population_change  Median_Age  \
0  Niagara Falls  New York     48460.0              -12.8        38.6   

   Median_Income  per_capita_Income  Median_House_Value  Median_Rent  \
0        35995.0            64894.0             76050.0        665.0   

   Cost_of_Living_Index  ...  PerCapIncscore  PopulationScore  EconomyScore  \
0                 100.7  ...            50.0        46.266667     22.666667   

  ClimateScore  CostofLivingScore  LivabilityScore  Heart Disease Value  \
0        52.33          61.183333        45.611667                215.1   

    county  id  code  
0  Niagara   0    NY  

[1 rows x 43 columns]

In [17]:
# Merge the columns on 'county' and 'code'

cleaned = df.merge(fips, how='left', on=['code', 'county'])
print(cleaned.columns)

Index(['City_Name', 'state', 'population', 'population_change', 'Median_Age',
       'Median_Income', 'per_capita_Income', 'Median_House_Value',
       'Median_Rent', 'Cost_of_Living_Index', 'Percent_below_Poverty',
       'Population_Density', 'Unemployment_rate', 'Most_Common_Industries',
       'Property_taxes', 'Latitude', 'Longitude', 'Average_Commute_Time',
       'AQI', 'popden_score', 'popden_norm', 'HDTI', 'HDTIscore',
       'HDTIscorenorm', 'RTI', 'RTIscore', 'RTIscorenorm', 'AQIscore',
       'MedAgescore', 'Povertyscore', 'COLIscore', 'popchangescore',
       'MedIncscore', 'PerCapIncscore', 'PopulationScore', 'EconomyScore',
       'ClimateScore', 'CostofLivingScore', 'LivabilityScore',
       'Heart Disease Value', 'county', 'id', 'code', 'fips'],
      dtype='object')


In [18]:
# Test engineer a feature showing percentage of people that died of Heart
# Disease (per 100,000) based on city population - data is given by county

rate = cleaned['Heart Disease Value'][0] / cleaned['population'][0]
cleaned['Heart Disease Value'][0] / rate

48460.0

In [19]:
# Actually engineer the features

cleaned['scaled_heart_disease_deaths'] = (cleaned['Heart Disease Value'] /
                                          100000) * cleaned['population']
cleaned['normalized_heart_disease'] = (cleaned['scaled_heart_disease_deaths'] /
                                       cleaned['population']) * 100

In [20]:
cleaned[:1]

City_Name     state  population  population_change  Median_Age  \
0  Niagara Falls  New York     48460.0              -12.8        38.6   

   Median_Income  per_capita_Income  Median_House_Value  Median_Rent  \
0        35995.0            64894.0             76050.0        665.0   

   Cost_of_Living_Index  ...  ClimateScore  CostofLivingScore  \
0                 100.7  ...         52.33          61.183333   

   LivabilityScore Heart Disease Value   county  id  code     fips  \
0        45.611667               215.1  Niagara   0    NY  36063.0   

   scaled_heart_disease_deaths  normalized_heart_disease  
0                    104.23746                    0.2151  

[1 rows x 46 columns]

In [21]:
# Check against the dataset where nulls are removed

print(cleaned.shape)
print('-------------')
print(cleaned.dropna().shape)

(1145, 46)
-------------
(1112, 46)


In [22]:
cleaned.describe()

population  population_change   Median_Age  Median_Income  \
count  1.145000e+03        1145.000000  1145.000000    1145.000000   
mean   4.752845e+04          39.893013    38.413362   67838.215721   
std    2.899021e+05         188.735034     6.634498   28989.195817   
min    5.858000e+03         -47.800000    16.100000   19478.000000   
25%    9.212000e+03           1.700000    34.400000   45764.000000   
50%    1.615000e+04           9.700000    37.900000   62058.000000   
75%    3.680200e+04          30.500000    42.400000   81630.000000   
max    8.622698e+06        4516.100000    76.500000  190942.000000   

       per_capita_Income  Median_House_Value  Median_Rent  \
count        1145.000000         1145.000000  1145.000000   
mean        65510.236681       325771.932751  1208.140611   
std         10002.205241       235058.932157   481.753270   
min         43529.000000        27400.000000   393.000000   
25%         59195.000000       153647.000000   840.000000   
50%         69117.000000       245385.000000  1120.000000   
75%         71805.000000       420800.000000  1465.000000   
max         80776.000000      1000000.000000  4008.000000   

       Cost_of_Living_Index  Percent_below_Poverty  Population_Density  ...  \
count           1145.000000            1145.000000         1145.000000  ...   
mean             106.044629              14.580611         3438.473362  ...   
std               25.306306               9.142488         4155.561578  ...   
min               70.200000               0.700000            6.000000  ...   
25%               86.700000               7.400000         1281.000000  ...   
50%               95.800000              12.700000         2349.000000  ...   
75%              121.200000              19.700000         4182.000000  ...   
max              199.900000              50.700000        55583.000000  ...   

       PopulationScore  EconomyScore  ClimateScore  CostofLivingScore  \
count      1145.000000   1145.000000   1145.000000        1145.000000   
mean         50.099272     49.000902     54.043170          49.646029   
std          11.695048     23.995842     31.171699          21.408295   
min          18.733333      2.566667      8.810000           0.243333   
25%          41.233333     29.366667     26.370000          34.400000   
50%          49.166667     49.500000     48.590000          51.366667   
75%          58.866667     68.666667     96.030000          66.816667   
max          89.200000     94.766667     97.840000          88.893333   

       LivabilityScore  Heart Disease Value           id          fips  \
count      1145.000000          1145.000000  1145.000000   1112.000000   
mean         50.697344           161.204716   572.000000  25215.169964   
std           9.059956            38.259544   330.677335  17538.273163   
min          20.932500            45.800000     0.000000   1001.000000   
25%          44.467500           135.500000   286.000000   6073.000000   
50%          50.547500           153.800000   572.000000  26065.000000   
75%          57.015000           179.200000   858.000000  42003.000000   
max          79.521667           332.700000  1144.000000  53077.000000   

       scaled_heart_disease_deaths  normalized_heart_disease  
count                  1145.000000               1145.000000  
mean                     72.764707                  0.161205  
std                     409.861943                  0.038260  
min                       3.370422                  0.045800  
25%                      15.592690                  0.135500  
50%                      26.212601                  0.153800  
75%                      54.661338                  0.179200  
max                   11821.718958                  0.332700  

[8 rows x 41 columns]

In [23]:
# Check for extreme outliers

cleaned[cleaned['scaled_heart_disease_deaths'] > 1000]

City_Name       state  population  population_change  Median_Age  \
46   Oklahoma City    Oklahoma    643648.0               27.2        34.2   
211     Fort Worth       Texas    874168.0               63.5        32.9   
364          Bronx    New York   1383871.0               20.3        31.2   
397       New York    New York   8622698.0                7.7        36.6   
940    Los Angeles  California   3999759.0                8.3        35.8   

     Median_Income  per_capita_Income  Median_House_Value  Median_Rent  \
46         52062.0            50051.0            158200.0        841.0   
211        60205.0            59206.0            169400.0        994.0   
364        37397.0            64894.0            400300.0       1187.0   
397        60879.0            64894.0            609500.0       1379.0   
940        60197.0            71805.0            647000.0       1397.0   

     Cost_of_Living_Index  ...  ClimateScore  CostofLivingScore  \
46                   85.4  ...         21.71          72.473333   
211                  94.3  ...         45.16          66.840000   
364                 177.3  ...         97.82           0.243333   
397                 162.7  ...         13.79           5.276667   
940                 145.8  ...         97.03           6.516667   

     LivabilityScore Heart Disease Value       county   id  code     fips  \
46         45.770833               207.8     Oklahoma   46    OK  40109.0   
211        53.666667               159.4      Tarrant  211    TX  48439.0   
364        44.099167               193.4        Bronx  364    NY  36005.0   
397        23.366667               137.1     New York  397    NY  36061.0   
940        47.736667               149.3  Los Angeles  940    CA   6037.0   

     scaled_heart_disease_deaths  normalized_heart_disease  
46                   1337.500544                    0.2078  
211                  1393.423792                    0.1594  
364                  2676.406514                    0.1934  
397                 11821.718958                    0.1371  
940                  5971.640187                    0.1493  

[5 rows x 46 columns]

In [29]:
# **July 17, 2020***
# Longitude is not a negative float therefore, it needs changed

cleaned['Longitude'] = cleaned['Longitude'].apply(lambda x: -x)

In [31]:
# Take subsets of the DataFrame so they can be entered as multiple tables
# in the Postgres instance

reference = cleaned[['id', 'code', 'fips', 'county',
                     'City_Name']]
heart = cleaned[['id', 'scaled_heart_disease_deaths',
                 'normalized_heart_disease']]
location = cleaned[['id', 'Latitude', 'Longitude']]
people_stats = cleaned[['id', 'Median_Age', 'population',
                        'population_change', 'Population_Density',
                        'popden_norm']]
economy = cleaned[['id', 'Median_Income', 'per_capita_Income',
                   'Percent_below_Poverty']]
job = cleaned[['id', 'Unemployment_rate', 'Most_Common_Industries',
               'Average_Commute_Time']]
housing = cleaned[['id', 'Median_House_Value', 'Median_Rent',
                   'Cost_of_Living_Index', 'Property_taxes']]

In [26]:
# The following instances of code must be uncommented to create CSV files

In [27]:
# reference.to_csv('..\\src\\datasets\\reference_data.csv', index=False)

In [28]:
# heart.to_csv('..\\src\\datasets\\heart_data.csv', index=False)

In [32]:
# location.to_csv('..\\src\\datasets\\location_data.csv', index=False)

In [30]:
# people_stats.to_csv('..\\src\\datasets\\people_stats_data.csv', index=False)

In [31]:
# economy.to_csv('..\\src\\datasets\\economy_data.csv', index=False)

In [32]:
# job.to_csv('..\\src\\datasets\\job_data.csv', index=False)

In [33]:
# housing.to_csv('..\\src\\datasets\\housing_data.csv', index=False)